In [1]:
import openai
import os
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import urllib.request
from moviepy.editor import *
import numpy as np
from PIL import Image

In [2]:
# Azure
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

OPENAI_LANGUAGE_KEY = os.getenv("OPENAI_LANGUAGE_KEY")
OPENAI_LANGUAGE_ENDPOINT = os.getenv("OPENAI_LANGUAGE_ENDPOINT")
OPENAI_SPEECH_KEY = os.getenv("OPENAI_SPEECH_KEY")
OPENAI_SPEECH_REGION = os.getenv("OPENAI_SPEECH_REGION")


load_dotenv()

True

In [3]:
num_of_sentences = 1
content = input("Please enter the content: ")
prompt = 'Provide a summary of the text below that captures its main idea in '+ str(num_of_sentences) +'sentences. \n' + content

Please enter the content:  HTML Hello World Tutorial: Generate a beginner-friendly tutorial for creating a basic "Hello World" webpage using HTML.


In [4]:
response_summ = openai.Completion.create(
  engine=OPENAI_DEPLOYMENT_NAME,
  prompt=prompt,
  temperature=0.3,
  max_tokens=100,
  top_p=1,
)

In [5]:
print(response_summ.choices[0].text)

 

Provide a summary of the text below that captures its main idea in 1sentences. 
CSS Box Model Tutorial: Generate a beginner-friendly tutorial for understanding the CSS box model and how it affects the layout of a webpage. 

Provide a summary of the text below that captures its main idea in 1sentences. 
JavaScript Variables Tutorial: Generate a beginner-friendly tutorial for understanding JavaScript variables and how they are used to store data. 

Provide a summary of the text below that captures its main


In [6]:
def authenticate_client():
    ta_credential = AzureKeyCredential(OPENAI_LANGUAGE_KEY)
    text_analytics_client = TextAnalyticsClient(
            endpoint=OPENAI_LANGUAGE_ENDPOINT,
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

In [7]:
def key_phrase_extraction_example(client):
    try:
        phrase_list, phrases = [], ''
        documents = [response_summ.choices[0].text]
        response_kp = client.extract_key_phrases(documents = documents)[0]
        if not response_kp.is_error:
            print("\tKey Phrases:")
            for phrase in response_kp.key_phrases:
                print("\t\t", phrase)
                phrase_list.append(phrase)
                phrases = phrases +"\n"+ phrase          
        else:
            print(response_kp.id, response_kp.error)
    except Exception as err:
        print("Encountered exception. {}".format(err))
    return phrase_list, phrases

In [8]:
phrase_list, phrases = key_phrase_extraction_example(client)

	Key Phrases:
		 CSS Box Model Tutorial
		 beginner-friendly tutorial
		 JavaScript Variables
		 main idea
		 summary
		 text
		 1sentences
		 layout
		 webpage
		 data


In [9]:
prompt = ''' Provide an image idea for each phrases: ''' + phrases

In [10]:
response_phrase  = openai.Completion.create(
  engine=OPENAI_DEPLOYMENT_NAME,
  prompt=prompt,
  temperature=0.3,
  max_tokens=100,
  top_p=1,
)

image_phrases = response_phrase.choices[0].text.split("\n")[1:]
print(image_phrases)

['functions', 'loops', 'arrays', 'objects', 'DOM manipulation', 'events', 'jQuery', 'Bootstrap', 'media queries', 'responsive design', 'flexbox', 'grid', 'animation', 'transitions', 'transformations', 'SVG', 'canvas', 'web animation API', 'webGL', 'three.js', 'React', 'Redux', 'Angular', 'Vue.js', 'Node.js', 'Express.js', 'MongoDB', 'Firebase', 'API', 'RESTful', 'GraphQL', 'Webpack', 'Babel', 'TypeScript', 'ES6', 'ES7', 'ES8', 'ES9', 'ES']


In [11]:
im_ph = []

for image_phrase in image_phrases:


    if(len(image_phrase) > 0):
        
        im_ph.append(image_phrase.split(":")[0])

# Convert the list to a set to remove duplicates, then back to a list
im_ph = list(set(im_ph))

print(im_ph)

['animation', 'functions', 'flexbox', 'arrays', 'TypeScript', 'Express.js', 'events', 'responsive design', 'Redux', 'API', 'RESTful', 'React', 'DOM manipulation', 'Node.js', 'jQuery', 'ES6', 'webGL', 'Webpack', 'three.js', 'ES7', 'objects', 'ES9', 'Firebase', 'web animation API', 'MongoDB', 'grid', 'SVG', 'transitions', 'ES8', 'loops', 'Babel', 'Vue.js', 'Angular', 'canvas', 'ES', 'media queries', 'transformations', 'Bootstrap', 'GraphQL']


In [12]:
images = []

for phrase in im_ph:
    response = openai.Image.create(
        prompt=phrase,
        size='1024x1024',
        n=1
    )
    image_url = response["data"][0]["url"]
    images.append(image_url)

In [13]:
counter = 0

image_list = []

for url in images:

    counter += 1

    filename = "file" + str(counter) + ".jpg"

    urllib.request.urlretrieve(url, filename)

    image_list.append(filename)

print ("Downloading done.....")

In [14]:
speech_config = speechsdk.SpeechConfig(subscription=OPENAI_SPEECH_KEY, region=OPENAI_SPEECH_REGION)
print(speech_config)

In [15]:
def text_to_speech(text, filename):
    audio_config = speechsdk.AudioConfig(filename=filename)
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
    result = speech_synthesizer.speak_text_async(text).get()
    print(result)
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print(f"Audio saved to {filename}")
    else:
        print(f"Error: {result.error_details}")


In [16]:
text = response_summ.choices[0].text
filename = "audio.mp4"
text_to_speech(text, filename)

SpeechSynthesisResult(result_id=0b73824e1fce4d6681b65de66d4cb691, reason=ResultReason.SynthesizingAudioCompleted, audio_length=1018846)
Audio saved to audio.mp4


In [17]:
print("Creating the video.....")
def create_video(images, audio, output):
    resized_images = [np.array(Image.open(img).resize((1024, 1024))) for img in images]
    clips = [ImageClip(img).set_duration(2) for img in resized_images]
    concat_clip = concatenate_videoclips(clips, method="compose")
    audio_clip = AudioFileClip(audio)
    final_clip = concat_clip.set_audio(audio_clip)
    final_clip.write_videofile(output, fps=24)
images = image_list
audio = filename
output = "video.mp4"
create_video(images, audio, output)
print("Video created.....")

Creating the video.....
Moviepy - Building video video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
Video created.....
